In this notebook we will create the Probabilistic analysis (probability distribution and heatmap) for each of the agent's mode and traffic profiles per slice.
在本笔记本中，我们将为每个代理的每个模式和每个切片的流量配置文件创建概率分析（概率分布和热图）。

In [2]:
import sys
sys.path.insert(0, '../../')

# # Import libraries
import pandas as pd # 数据处理库，用于数据框操作和数据分析
import numpy as np # 数值计算库，提供数学函数和数组操作
import matplotlib.pyplot as plt # 绘图库，用于创建静态图表和可视化
import networkx as nx # 复杂网络分析库，用于构建和可视化图结构
import ast
import os
from graphviz import Digraph  # 图形可视化库，用于绘制有向图
%matplotlib inline
import plotly.graph_objects as go # 交互式绘图库，创建动态可交互的图表
from plotly.subplots import make_subplots # # 创建多个子图的工具
from IPython.display import display # Jupyter显示工具，用于更好地展示DataFrame等对象
import seaborn as sns # 基于matplotlib的统计可视化库，提供更美观的图表样式

# Import Constatns - 导入项目中定义的常量
from script.experiments_constants import AGENT_EXPERIMENT_INFO
from script.experiments_constants import ENV_KPI_NAME_LIST

# 导入项目地址常量，定义数据存储和项目文件的根路径
from constants import PROJ_ADDR


# 导入项目自定义模块
from script.load_data import handle_data
# 导入数据加载函数，负责：
# 1. 检查数据缓存
# 2. 从原始日志文件加载数据
# 3. 返回清洗后的KPI数据和决策数据
# 使用示例：kpi_data, decision_data = handle_data(agent_info, user_number=6)
from script.symbolic_representation import create_symbolic_state_decision_matrix
# 导入符号化表示创建函数，这是SYMBXRL方法的核心
# 功能：将数值状态和决策转换为基于一阶逻辑的符号表示
# 论文应用：实现从连续数值空间到离散符号空间的转换

# Load and Show some results for each one of the agents

In this part we will load the data of one of the agents and visualize some of the numerical data.

加载并展示每个代理的一些结果
在本部分中，我们将加载其中一个代理的数据，并对部分数值数据进行可视化处理。

## Graph Plotter Function Code【EE-概率分布】【和 1.ipynb一样】
绘图函数代码

In [ ]:
# Choose one of the agents: 'embb-trf1', 'embb-trf2', 'urllc-trf1', 'urllc-trf2'
# 选择一个智能体类型进行分析
agent = 'embb-trf2'
# 这里选择了"embb-trf2" - 增强移动宽带流量类型2
# 论文意义：专注于分析eMBB（增强移动宽带）类型的网络切片行为
# 可选值说明：
# - 'embb-trf1': 增强移动宽带 - 流量类型1
# - 'embb-trf2': 增强移动宽带 - 流量类型2
# - 'urllc-trf1': 超可靠低延迟通信 - 流量类型1
# - 'urllc-trf2': 超可靠低延迟通信 - 流量类型2
# 这些代表不同的网络服务类型和流量特性

# Choose one of the users: 3, 4, 5, 6 - Combination of agent and user will determine a specific experiment
# 选择用户数量：3, 4, 5, 6 - 智能体和用户数量的组合唯一确定一个实验场景
user = 6
# 注释说明：每个(agent, user)组合对应一组特定的实验数据

# Assuming AGENT_EXPERIMENT_INFO, handle_data and create_symbolic_state_decision_matrix are defined elsewhere in your codebase
# 假设AGENT_EXPERIMENT_INFO、handle_data和create_symbolic_state_decision_matrix已在代码库的其他地方定义
# 这些模块和函数通过前面的import语句导入

# 获取选定智能体的详细实验配置信息
agent_info = AGENT_EXPERIMENT_INFO[agent]
# 加载和处理实验数据 - 数据预处理阶段
kpi_data, decision_data = handle_data(agent_info, user)
# 创建符号化状态决策矩阵 - SYMBXRL核心算法步骤
symbolic_df, marker_df = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, user)
# 将PRB决策和调度决策合并成一张表，便于后续分析【大csv表格(符号化后的kpi+决策)】
symbolic_df['combined_decision'] = symbolic_df.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)

# 打印当前分析场景信息
print(f"Agent: {agent} - Users: {user}")
print("")  # 输出空行，使控制台输出更清晰易读

In [ ]:
symbolic_df # 打印表格

## Plot the Probability Distributino of agent's effects on the environment
绘制智能体对环境影响的概率分布图

In [ ]:
# 定义创建效果列表的函数
def create_effects_list(kpis=['tx_brate', 'tx_pckts', 'dl_buffer'], changes=['dec', 'const', 'inc']):
    """
    生成所有可能的符号化效果组合
    论文应用：系统地枚举所有可能的KPI变化模式，用于后续的概率分布分析
    """
    return {
        kpi: [f'{change}({kpi}, Q{quartile})' for quartile in range(1, 5) for change in changes] for kpi in kpis
    }
 # 列表推导式解析：
    # 对于每个KPI，生成所有变化类型和分位区间的组合
    # 输出示例：['dec(tx_brate, Q1)', 'const(tx_brate, Q1)', 'inc(tx_brate, Q1)',
    #           'dec(tx_brate, Q2)', 'const(tx_brate, Q2)', 'inc(tx_brate, Q2)', ...]
    # 每个KPI生成：3种变化 × 4个分位区间 = 12种效果


# Define slices and KPIs
# 定义切片和KPI列表
slices = [0, 1, 2]  # 三个网络切片ID
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']  # 要分析的三个关键性能指标
effects_list = create_effects_list()  # 生成效果列表


# Color Universal Design (CUD) color scheme for color blind friendliness
# 使用色盲友好配色方案 - 色彩通用设计(CUD)
colors = ["#E69F00", "#56B4E9", "#009E73"]  # 橙色, 天蓝色, 蓝绿色
# 论文意义：确保图表对不同色觉类型的人都可读，提高可访问性

# Create a 1x3 subplot figure
# 创建1行3列的子图，设置图形大小，共享y轴
fig, axes = plt.subplots(1, len(kpis), figsize=(18, 6), sharey=True)
# 参数说明：
# - 1, len(kpis): 1行，列数等于KPI数量(3)
# - figsize=(18,6): 图形宽18英寸，高6英寸
# - sharey=True: 所有子图共享y轴刻度，便于比较

# 设置整个图形的主标题
fig.suptitle(f'Effect Probability Distribution - {agent} - user:{user}', fontsize=20)
# 显示当前分析的智能体类型和用户数量



# 设置柱状图参数
bar_width = 0.2  # 每个柱子的宽度
x = np.arange(len(effects_list[kpis[0]]))  # x轴位置：[0, 1, 2, ..., 11] (12个效果)

# 遍历每个KPI子图
for i, kpi in enumerate(kpis): # i: 子图索引, kpi: 当前KPI名称
    # 遍历每个切片，在同一个子图中绘制不同切片的柱状图
    for j, slice_id in enumerate(slices):  # j: 切片索引, slice_id: 当前切片ID
        # Filter data for the current slice
        # 过滤出当前切片的数据
        slice_data = symbolic_df[symbolic_df['slice_id'] == slice_id]
        
        # Calculate the effect probability
        # 计算当前切片的效果概率分布
        effect_counts = slice_data[kpi].value_counts(normalize=True).reindex(effects_list[kpi], fill_value=0)
        # 分步解析：
        # 1. slice_data[kpi]: 选择当前切片的KPI符号数据
        # 2. .value_counts(normalize=True): 计算每种符号的频率并归一化为概率
        # 3. .reindex(effects_list[kpi], fill_value=0): 重新索引，确保所有12种效果都有值，缺失的填0
        
        # Plot the probability distribution as a bar plot
         # 绘制柱状图
        if i == 0:  # Only add the label once for the legend # 只在第一个子图添加图例标签（避免重复）
            axes[i].bar(x + j * bar_width,       # x位置：基础位置 + 切片偏移
                       effect_counts.values,     # 高度：概率值
                       bar_width,                # 宽度
                       label=f'Slice {slice_id}', # 图例标签
                       color=colors[j])          # 使用预定义颜色
        else:
            axes[i].bar(x + j * bar_width,
                       effect_counts.values,
                       bar_width,
                                                # 其他子图不重复添加标签
                       color=colors[j])
    
    # Set titles and labels
     # 设置每个子图的标题和标签
    axes[i].set_title(f'{kpi}', fontsize=18)  # 子图标题显示KPI名称
    axes[i].set_xlabel('Effect', fontsize=16)  # x轴标签
    axes[i].set_xticks(x + bar_width)  # 设置x轴刻度位置
    axes[i].set_xticklabels(effects_list[kpi], rotation=45, ha='right')  # 设置x轴标签，旋转45度，右对齐
    axes[i].tick_params(axis='both', which='major', labelsize=14)  # 设置刻度标签字体大小
    axes[i].grid(True)  # 显示网格线，便于读数

# Set a common y-axis label
# 设置共同的y轴标签（只在第一个子图显示）
axes[0].set_ylabel('Probability', fontsize=16)


# Add a legend
# 添加图例到整个图形
fig.legend(loc='upper right', fontsize=14)  # 图例位置在右上角

# Adjust the subplot layout to make room for the rotated labels
# 调整子图布局，为旋转的标签留出空间
plt.tight_layout()  # 自动调整子图参数
fig.subplots_adjust(top=0.85, bottom=0.25)  # 手动调整：顶部留85%，底部留25%空间

# Display the plot
# 显示图形
plt.show()

# Save the figure as PDF
# 保存图形为PDF文件【目录：SYMBXRL\A1-NetworkSlicingSchedulingPolicy\results\Probabilistic_Analysis\prob_dist\embb-trf1-users3】
output_dir = f'{proj_address}/A1-NetworkSlicingSchedulingPolicy/results/Probabilistic_Analysis/prob_dist/{agent}-users{user}'  # Specify the directory to save the graphs  # 指定保存目录
# 创建目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)
 # 文件名
file_name = f'Effect_Probability_Distribution_{agent}_user_{user}.pdf'
# 完整文件路径
pdf_file_path = os.path.join(output_dir, file_name)
# 保存为PDF，紧凑边界
fig.savefig(pdf_file_path, format='pdf', bbox_inches='tight')

# 打印保存路径
print(f"Figure saved as {pdf_file_path}")

## Plot the heat map of decision and their effects on the environment【EE-热力图分析】
绘制决策及其对环境影响的热力图

In [ ]:
# 定义创建效果列表的函数
def create_effects_list(kpis=['tx_brate', 'tx_pckts', 'dl_buffer'], changes=['dec', 'const', 'inc']):
    """
    生成所有可能的符号化效果组合
    论文应用：系统枚举所有KPI变化模式，为热力图提供完整的维度
    """
    return {
        kpi: [f'{change}({kpi}, Q{quartile})' for quartile in range(1, 5) for change in changes] for kpi in kpis
    }

effects_list = create_effects_list()  # 生成效果列表
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']  # 三个关键性能指标
slices = [0, 1, 2]  # 三个网络切片

# 绘制决策效果概率热力图并保存为pdf
def plot_heatmaps(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    """
        绘制决策效果概率热力图
        论文应用：可视化展示不同决策导致的各种效果的概率分布
     """
    # Concatenate all effects
    # 合并所有效果，创建完整的效果列表
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]
     # 示例：['dec(tx_brate,Q1)', 'const(tx_brate,Q1)', ..., 'dec(dl_buffer,Q4)', ...]
    # 总共：3个KPI × 12种效果 = 36个效果维度


    cmap = 'viridis'  # 使用viridis颜色映射（色盲友好且美观）

    def format_value(val):
        """
        格式化显示值：只显示显著的概率值（≥0.5%）
        论文意义：避免图表过于拥挤，突出重要信息
        """
        if val >= 0.005:  # Threshold set to 0.005 # 阈值设为0.005（0.5%）
            return f'{val * 100:.1f}%'  # Show as percentage with one decimal place # 显示为百分比，保留一位小数
        return '' # 小于0.5%的概率不显示数字

     # 创建3行1列的子图，每个切片一个热力图
    # Create a 3x1 subplot figure
    fig, axes = plt.subplots(len(slices), 1, figsize=(20, 15))  # 参数说明：3行1列，图形大小20×15英寸
    # 标题
    fig.suptitle(f'Effect Probabilities Heatmaps for Agent {agent} - user {user}', fontsize=16)

    # 遍历每个切片，绘制对应的热力图
    for idx, slice_id in enumerate(slices):
        # 过滤出当前切片的数据
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]

        # Get unique decisions for this slice
        # 获取独属于该切片的决策并排序（按出现频率）
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts() # 计算每个决策的出现次数
        sorted_slice_decisions = decision_counts.index.tolist() # 按频率排序的决策列表

        # Calculate the height based on the number of decisions in this slice
        # 动态计算图形高度（基于决策数量）
        decision_height = 0.3  # Height per decision in inches # 每个决策占0.3英寸高度
        min_height = 5  # Minimum height of the plot # 最小高度
        calculated_height = max(min_height, len(sorted_slice_decisions) * decision_height)

        # Initialize the matrix for all KPIs and decisions in this slice
        # 初始化概率矩阵：决策数 × 效果数
        div_matrix = np.zeros((len(sorted_slice_decisions), len(all_effects)))
        # 矩阵结构：行=决策，列=效果，值=概率

        # 填充概率矩阵
        for i, decision in enumerate(sorted_slice_decisions):
            # 过滤出采用当前决策的所有数据
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0 # 列索引指针
                 # 遍历每个KPI类型
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]  # 当前KPI的所有可能效果
                     # 计算当前决策下各种效果的出现次数
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    # 将效果计数填入矩阵的对应位置
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects) # 移动列索引指针

        # Normalize the matrix for this slice
        # 归一化矩阵：将计数转换为概率
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        # 验证概率总和是否为1（或非常接近1由于浮点数精度舍入）
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        # 创建注释数组：只显示显著的概率值
        annot = np.vectorize(format_value)(div_matrix)
         # np.vectorize将format_value函数向量化，应用于矩阵每个元素

        # Plot heatmap with fixed scale from 0 to 1 and display formatted values
         # 使用seaborn绘制热力图
        sns.heatmap(div_matrix, ax=axes[idx], cmap=cmap, vmin=0, vmax=1, cbar=True, 
                    annot=annot, fmt='', annot_kws={'size': 6}, linewidths=0, linecolor='white')
        # 参数说明：
        # - vmin=0, vmax=1: 固定颜色映射范围为0到1
        # - annot=annot: 显示格式化后的注释文本
        # - fmt='': 注释文本已经是格式化好的字符串
        # - linewidths=0: 单元格之间无间隔线


        # 设置子图标题和标签
        axes[idx].set_title(f'Slice {slice_id}', fontsize=14)
        axes[idx].set_xlabel('Effects', fontsize=12)
        axes[idx].set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
         # 设置x轴刻度和标签（效果名称）
        axes[idx].set_xticks(np.arange(len(all_effects)) + 0.5)  # 刻度位置居中
        axes[idx].set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=10)
        # rotation=45: 标签旋转45度避免重叠
        
        # 设置y轴刻度和标签（决策名称）
        axes[idx].set_yticks(np.arange(len(sorted_slice_decisions)) + 0.5)
        axes[idx].set_yticklabels(sorted_slice_decisions, rotation=0, fontsize=10)

    # Adjust layout
    # 调整布局，为主标题留出空间
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the figure as PDF
    # 保存图形为PDF
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True) # 创建目录（如果不存在）
    fig.savefig(output_path, format='pdf', bbox_inches='tight') # 紧凑边界保存

    # Display the plot
    # 显示图形
    plt.show()


# Plot the heatmaps for the specified agent and user and save as PDF
# 调用函数 绘制热力图并保存为PDF【SYMBXRL\A1-NetworkSlicingSchedulingPolicy\results\Probabilistic_Analysis\heatmaps\embb-trf1-users3\Effect_Probabilities_Heatmaps_embb-trf1_user_3.pdf】
output_file = f'{proj_address}/A1-NetworkSlicingSchedulingPolicy/results/Probabilistic_Analysis/heatmaps/{agent}-users{user}/Effect_Probabilities_Heatmaps_{agent}_user_{user}.pdf'
plot_heatmaps(symbolic_df, agent, effects_list, kpis, slices, output_file)

print(f"Figure saved as {output_file}")